In [1]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2024-04-27 18:40:23,817 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle d41711823afc7a268e68488fc562e5a3 initialized by task ('shuffle-transfer-d41711823afc7a268e68488fc562e5a3', 81) executed on worker tcp://127.0.0.1:53845
2024-04-27 18:47:17,720 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle d41711823afc7a268e68488fc562e5a3 deactivated due to stimulus 'task-finished-1714236437.717788'
Task exception was never retrieved
future: <Task finished name='Task-958349' coro=<Client._gather.<locals>.wait() done, defined at /Users/stefanotavonatti/Teaching/lab5/venv3/lib/python3.11/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/stefanotavonatti/Teaching/lab5/venv3/lib/python3.11/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit


In [2]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv', 
                 storage_options={'token': 'anon'},
                parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], dtype={'tolls_amount': 'float64'}
                )

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [5]:
df.dtypes

VendorID                           int64
tpep_pickup_datetime      datetime64[ns]
tpep_dropoff_datetime     datetime64[ns]
passenger_count                    int64
trip_distance                    float64
pickup_longitude                 float64
pickup_latitude                  float64
RateCodeID                         int64
store_and_fwd_flag       string[pyarrow]
dropoff_longitude                float64
dropoff_latitude                 float64
payment_type                       int64
fare_amount                      float64
extra                            float64
mta_tax                          float64
tip_amount                       float64
tolls_amount                     float64
improvement_surcharge            float64
total_amount                     float64
dtype: object

Remember to `.persist()` your data if you are going to use it more than once

```python
df = df.persist()
```

In [7]:
df_no_dates = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv', 
                 storage_options={'token': 'anon'},
                )

In [8]:
df_grouped = df_no_dates.groupby("VendorID").agg({"VendorID": "count", "tip_amount": "max", "total_amount": "sum"}).compute()
df_grouped

,VendorID,tip_amount,total_amount
VendorID,,,
2,76658633,3950588.80,1.245614e+09
1,69454356,1603.05,1.106257e+09


In [10]:
type(df_grouped)

pandas.core.frame.DataFrame

In [24]:
df_grouped.idxmax(axis="index")

VendorID        2
tip_amount      2
total_amount    2
dtype: int64

2. **Find the vendor with the highest number of trips**

In [31]:
df_grouped.filter(items=[df_grouped.idxmax(axis="index")["VendorID"]], axis=0)

,VendorID,tip_amount,total_amount
2,76658633,3950588.8,1.245614e+09


3. **Find the vendor with the highest tip**

In [32]:
df_grouped.filter(items=[df_grouped.idxmax(axis="index")["tip_amount"]], axis=0)

,VendorID,tip_amount,total_amount
2,76658633,3950588.8,1.245614e+09


4. **Find the vendor that made the most money and how much money it made**

In [33]:
df_grouped.filter(items=[df_grouped.idxmax(axis="index")["total_amount"]], axis=0)

,VendorID,tip_amount,total_amount
2,76658633,3950588.8,1.245614e+09


5. **Find the week day with the highest tip**
6. **Find the week day with the highest number of runs**

In [10]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv', 
                 storage_options={'token': 'anon'},
                parse_dates=['tpep_pickup_datetime'], dtype={'tolls_amount': 'float64'}
                )

In [11]:
df_tips = df[["tpep_pickup_datetime", "tip_amount"]]
df_tips

,tpep_pickup_datetime,tip_amount
npartitions=353,,
,datetime64[ns],float64
,...,...
...,...,...
,...,...
,...,...


In [12]:
df_tips["week_day"] = df_tips["tpep_pickup_datetime"].dt.dayofweek
df_tips

,tpep_pickup_datetime,tip_amount,week_day
npartitions=353,,,
,datetime64[ns],float64,int32
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [13]:
df_tips.head()

,tpep_pickup_datetime,tip_amount,week_day
0,2015-01-15 19:05:39,3.25,3
1,2015-01-10 20:33:38,2.00,5
2,2015-01-10 20:33:38,0.00,5
3,2015-01-10 20:33:39,0.00,5
4,2015-01-10 20:33:39,0.00,5


In [14]:
df_tips_grouped = df_tips.groupby("week_day").agg({"week_day": "count", "tip_amount": "max"}).compute()
df_tips_grouped

2024-04-27 18:41:06,332 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-04-27 18:41:06,412 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-04-27 18:41:13,765 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-04-27 18:41:13,843 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-04-27 18:41:13,920 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-04-27 18:41:14,011 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-04-27 18:41:20,466 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-04-27 18:41:20,529 - distributed.utils_perf - WARNING - full garbage collections took

,week_day,tip_amount
week_day,,
5,22608347,823.63
0,18667030,886.64
3,21865569,1603.05
2,21076056,910.05
1,20129034,980.91
6,19794851,3950588.80
4,21972102,969.69


In [15]:
df_tips_grouped.idxmax(axis="index")

week_day      5
tip_amount    6
dtype: int32